In [112]:
# imports and options
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

In [113]:
# read timelines and match ids and tournament ids
timelines_df = pd.read_csv('../data/match_timelines.csv')
team_names_df = pd.read_csv('../data/match_ids.csv')
tournaments_df = pd.read_csv('../data/tournament_ids.csv')

# select correct tournaments
tournaments_df = tournaments_df[tournaments_df['tournament_id'].isin([57, 58, 59, 60, 61, 62, 102, 103, 104, 105, 106, 107, 152, 153, 154, 155, 156, 157])]

In [114]:
# add column of age group and gender
tournaments_df['age'] = ['F12', 'M12', 'F13', 'F14', 'M13', 'M14',
                         'M13', 'F13', 'F12', 'M12', 'F14', 'M14',
                         'M13', 'F12', 'F13', 'F14', 'M14', 'M12']

# create year column from name
tournaments_df['year'] = tournaments_df['name'].str.extract(r'(\d{4})')

In [115]:
# list international teams
int_teams = ['Nords', 'jylland', 'Shamrock', 'Hamma', 'ndby', 'Vitesse', 'Bromma', 'Liding']

In [116]:
# create boolean variables for international teams
team_names_df['home_team_int'] = team_names_df['home_team'].apply(lambda x: 1 if any(sub in x for sub in int_teams) else 0)
team_names_df['away_team_int'] = team_names_df['away_team'].apply(lambda x: 1 if any(sub in x for sub in int_teams) else 0)

In [117]:
# add home and away team names to main dataframe
timelines_df = timelines_df.merge(team_names_df, on=['tournament_id', 'match_id'], how='left')

# add year and age group to main dataframe
timelines_df = timelines_df.merge(tournaments_df.drop('name', axis=1), on=['tournament_id'], how='left')

# convert times to datetime objects
# Convert to datetime
timelines_df['start_time'] = pd.to_datetime(timelines_df['start_time'], format='mixed')
timelines_df['end_time'] = pd.to_datetime(timelines_df['end_time'], format='mixed')

In [118]:
# Function to calculate Euclidean distance
def euclidean_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Apply distance calculation for "Pass" and "Dribble" events
timelines_df['distance'] = np.where(
    timelines_df['event_type'].isin(['Pass', 'Dribble']), 
    euclidean_distance(timelines_df['start_position_x'], timelines_df['start_position_y'], timelines_df['end_position_x'], timelines_df['end_position_y']),
    np.nan  # Assign NaN for other events
)

# calculate duration of event for possible events
timelines_df['duration'] = timelines_df['end_time'] - timelines_df['start_time']
timelines_df['duration'] = timelines_df['duration'].dt.total_seconds() #convert to seconds

# calculate speed for possible events
timelines_df['speed'] = np.where(
    timelines_df['event_type'].isin(['Pass', 'Dribble']), 
    timelines_df['distance'] / timelines_df['duration'],
    timelines_df['speed']
)

# create column of international level (0=f-f, 1=i-f, 2=i-i)
timelines_df['int_level'] = timelines_df['home_team_int'] | timelines_df['away_team_int']

In [119]:
# transform second period into an extension of the first period

# remove matches where there is no first period
valid_matches = timelines_df[timelines_df['period'] == 1]['match_id'].unique()
timelines_df = timelines_df[timelines_df['match_id'].isin(valid_matches)]

# timestamp of last event of first period
first_period_durations = timelines_df[timelines_df['period'] == 1].groupby('match_id')['seconds_from_period_start'].max()

# transform second period timestamps
def transform_time(row):
    if row['period'] == 2:
        return row['seconds_from_period_start'] + first_period_durations[row['match_id']]
    return row['seconds_from_period_start']

timelines_df['seconds_from_period_start'] = timelines_df.apply(transform_time, axis=1)

# drop the 'period' column as it is no longer needed
# timelines_df = timelines_df.drop(columns=['period'])

In [120]:
# pd.options.display.max_rows = 900
# timelines_df[timelines_df['match_id'] == 1346]

### Processing

In [121]:
import matplotlib.pyplot as plt

# match ids and group into groups of 50
match_ids = timelines_df['match_id'].unique()
match_groups = [match_ids[i:i + 50] for i in range(0, len(match_ids), 50)]

# plot 

# count occurrences of each event_type per match a few matches at a time
# for matches in match_groups:
#     event_counts = timelines_df[timelines_df['match_id'].isin(matches)].pivot_table(index='match_id', columns='event_type', aggfunc='size', fill_value=0)
#     event_counts.plot(kind='bar', stacked=True, figsize=(15, 6), colormap='viridis')

#     plt.xlabel("Match ID")
#     plt.ylabel("Event Count")
#     plt.title("Event Type Distribution per Match")
#     plt.legend(title="Event Type", bbox_to_anchor=(1.05, 1), loc='upper left')
#     plt.show()

#### Remove matches that are most likely faulty

In [122]:
# group by matches
df_matches = timelines_df.groupby('match_id')

# get matches with time gaps between events of over 50 seconds
threshold = 50

# Compute time gaps
timelines_df['time_gap'] = df_matches['seconds_from_period_start'].diff()

# Identify matches with gaps above threshold
timelines_df['gap_exceeds_threshold'] = timelines_df['time_gap'] > threshold
timelines_df['gap_exceeds_threshold_300'] = timelines_df['time_gap'] > 300

In [123]:
# get matches where two consecutive events are kick-offs
timelines_df['consecutive_kickoffs'] = (timelines_df['event_type'] == 'KickOff') & (timelines_df['event_type'].shift(1) == 'KickOff')

In [124]:
# get matches with under 300 events
event_counts = timelines_df.groupby('match_id').size().reset_index(name='event_count')
too_few_events_ids = event_counts['match_id'][event_counts['event_count'] < 300]

In [125]:
# remove matches with time gaps of over five minutes and matches with less than 300 events
remove_match_indices = timelines_df[(timelines_df['gap_exceeds_threshold_300'] == True)]['match_id'].unique()
remove_match_indices = list(set(remove_match_indices) | set(too_few_events_ids))
timelines_df = timelines_df[~timelines_df['match_id'].isin(remove_match_indices)].copy()

In [126]:
# scale F12 dimensions to the same dimensions as other age groups, since distances are calculated already,
# these won't affect the speed calculations

length_ratio = 100/63
width_ratio = 63/40

age_group_mask = timelines_df['age'] == 'F12'
timelines_df.loc[age_group_mask, ['start_position_x', 'end_position_x']] = timelines_df.loc[age_group_mask, ['start_position_x', 'end_position_x']]*length_ratio
timelines_df.loc[age_group_mask, ['start_position_y', 'end_position_y']] = timelines_df.loc[age_group_mask, ['start_position_y', 'end_position_y']]*width_ratio

In [127]:
# some matches are not normalized (meaning that home team always attacks to the right),
# check which matches these are and normalize them

df_passes = timelines_df[timelines_df['event_type'] == 'Pass']

pos_vars = ['start_position_x', 'end_position_x', 'start_position_y', 'end_position_y']

for match_id in df_passes['match_id'].unique():
    match_id_mask_1 = (timelines_df['match_id'] == match_id) & (timelines_df['period'] == 1)
    match_id_mask_2 = (timelines_df['match_id'] == match_id) & (timelines_df['period'] == 2)
    match_home_mask_1 = (df_passes.match_id == match_id) & (df_passes.team == 'Home') & (df_passes.period == 1)
    pass_dir_avg_x_1 = (df_passes.loc[match_home_mask_1, 'end_position_x'] - df_passes.loc[match_home_mask_1, 'start_position_x']).mean()
    match_home_mask_2 = (df_passes.match_id == match_id) & (df_passes.team == 'Home') & (df_passes.period == 2)
    pass_dir_avg_x_2 = (df_passes.loc[match_home_mask_2, 'end_position_x'] - df_passes.loc[match_home_mask_2, 'start_position_x']).mean()


    match_away_mask_1 = (df_passes.match_id == match_id) & (df_passes.team == 'Away') & (df_passes.period == 1)
    a_pass_dir_avg_x_1 = (df_passes.loc[match_away_mask_1, 'end_position_x'] - df_passes.loc[match_away_mask_1, 'start_position_x']).mean()
    match_away_mask_2 = (df_passes.match_id == match_id) & (df_passes.team == 'Away') & (df_passes.period == 2)
    a_pass_dir_avg_x_2 = (df_passes.loc[match_away_mask_2, 'end_position_x'] - df_passes.loc[match_away_mask_2, 'start_position_x']).mean()
        
    if pass_dir_avg_x_1 < 0 and a_pass_dir_avg_x_1 > 0:
        timelines_df.loc[match_id_mask_1, pos_vars] = timelines_df.loc[match_id_mask_1, pos_vars]*(-1)
    if pass_dir_avg_x_2 < 0 and a_pass_dir_avg_x_2 > 0:
        timelines_df.loc[match_id_mask_2, pos_vars] = timelines_df.loc[match_id_mask_2, pos_vars]*(-1)

In [128]:
# we can assume that no shots were taken from over 60 meters out, and these are mislabeled so we can correct them
timelines_df.loc[(timelines_df['event_type'] == 'Shot') & (timelines_df['team'] == 'Home') & (timelines_df['start_position_x'] < -10), 'team'] = 'Away'
timelines_df.loc[(timelines_df['event_type'] == 'Shot') & (timelines_df['team'] == 'Away') & (timelines_df['start_position_x'] > 10), 'team'] = 'Home'

In [129]:
# fix known missed labels of Goals, GoalKicks are also misslabeled but these can be ignored
mask1 = (timelines_df['match_id'] == 1298) & (timelines_df['seconds_from_period_start'] == 2072) & (timelines_df['event_type'] == 'Shot')
timelines_df.loc[mask1, 'team'] = 'Home'
mask2 = (timelines_df['match_id'] == 2290) & (timelines_df['seconds_from_period_start'] == 2053) & (timelines_df['event_type'] == 'Shot')
timelines_df.loc[mask2, 'team'] = 'Home'
mask3 = (timelines_df['match_id'] == 1258) & (timelines_df['seconds_from_period_start'] == 1518) & (timelines_df['event_type'] == 'Shot')
timelines_df.loc[mask3, 'team'] = 'Away'
mask4 = (timelines_df['match_id'] == 1307) & (timelines_df['seconds_from_period_start'] == 1117) & (timelines_df['event_type'] == 'Shot')
timelines_df.loc[mask4, 'team'] = 'Away'
mask5 = (timelines_df['match_id'] == 1304) & (timelines_df['seconds_from_period_start'] == 511) & (timelines_df['event_type'] == 'Shot')
timelines_df.loc[mask5, 'team'] = 'Home'

In [130]:
# remove the unneeded columns
timelines_df = timelines_df.drop(['gap_exceeds_threshold', 'gap_exceeds_threshold_300', 'consecutive_kickoffs'], axis='columns')

In [131]:
# save dataframe to csv
timelines_df.to_csv('../data/processed_timelines.csv', index=False)